In [124]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import mysql.connector as conn
import re
import statsmodels.api as sm

import sqlalchemy as sa
from sqlalchemy import create_engine

import json
import glob
import warnings
warnings.filterwarnings('ignore')

In [2]:
username = None
password = None

with open("../connection.json") as connection_file:
    cf = json.load(connection_file)
    username = cf["user"]
    password = cf["password"]

connection_uri = sa.engine.URL.create(
    drivername="mysql+pymysql",
    username=username,
    password=password,
    host="localhost",
    database="foodaps",
)

cnx = conn.connect(
    host="localhost",
    database="foodaps",
    user=username,
    password=password,
)

In [185]:
AME_QUERY_FILE = "../sql/ame.sql"
ENU_QUERY_FILE = "../sql/enu.sql"
INDIV_CHAR_QUERY_FILE = "../sql/indiv_char.sql"
HOUSEHOLD_CHAR_QUERY_FILE = "../sql/household_char.sql"
FAH_NUTRIENTS_QUERY_FILE = "../sql/fah_nutrients.sql"

enu_weights = {
    "breakfast": 0.16, 
    "lunch": 0.43, 
    "dinner": 0.30, 
    "snack": 0.11
}

def get_query(conn, query_file): 
    """
    Given a query and a connection returns a pandas table. 
    """
    with open(query_file, "r") as f:
        query = f.read()
    return pd.read_sql(query, conn)

def create_enu_table(): 
    """
    Creates a table from two queries to calculate the 
    equivalent nutritional units (ENU) for each person in the
    houshold and aggregates it to the household level. Does 
    not account for guests. 
    """
    ame = get_query(cnx, AME_QUERY_FILE)
    enu = get_query(cnx, ENU_QUERY_FILE)

    enu["tot_snacks"] = enu["tot_snackam"] + enu["tot_snackpm"]
    enu["weekly_enu"] = (enu["tot_brkfst"]*enu_weights["breakfast"] 
        + enu["tot_lunch"]*enu_weights["lunch"] 
        + enu["tot_dinner"]*enu_weights["dinner"] 
        + enu["tot_snacks"]*enu_weights["snack"])

    return ame.merge(enu, left_on=["HHNUM", "PNUM"], right_on=["HHNUM", "PNUM"], how="left").groupby(
        "HHNUM").agg({
            "weekly_enu": "sum",
            "TEE": "sum", 
            "AME": "sum"
        }).reset_index()

def majority_vote_household_demographic(hh_series):
    """
    If a household comprises of members of the same 
    race, then returns that race. Otherwise, returns
    "mixed".
    """
    if hh_series.unique().shape[0] == 1:
        return hh_series.unique()[0]
    else:
        return "mixed"

def create_household_table():
    """
    Creates a table from two queries to create household
    characteristics for explanatory analysis. 
    """
    indiv = get_query(cnx, INDIV_CHAR_QUERY_FILE)
    hh = get_query(cnx, HOUSEHOLD_CHAR_QUERY_FILE)

    hh["guest_enu"] = (hh["nguest_brkfst"] * enu_weights["breakfast"] + 
        hh["nguest_lunch"] * enu_weights["lunch"] + 
        hh["nguest_dinner"] * enu_weights["dinner"] + 
        hh["nguest_snack"] * enu_weights["snack"])

    hh_demo = indiv.groupby("hhnum").agg({
        "Race": majority_vote_household_demographic,
        "Hispanic": majority_vote_household_demographic,
        "female_head": "first"}).reset_index()

    return hh.merge(hh_demo, left_on="hhnum", right_on="hhnum", how="left")

def create_nutrients_table(): 
    """
    Creates a table from a query for weekly nutrition by household. 
    """
    fah_nut = get_query(cnx, FAH_NUTRIENTS_QUERY_FILE)
    fah_nut = fah_nut.dropna()

    #remove SUM() and hh_ from column names
    fah_nut.columns = [col.split("_")[1] if "SUM" in col else col for col in fah_nut.columns]

    #remove ) from column names
    fah_nut.columns = [col.split(")")[0] if ")" in col else col for col in fah_nut.columns]

    #add fah to column_names except for hhnum
    fah_nut.columns = ["fah_" + col if col != "hhnum" else col for col in fah_nut.columns]

    return fah_nut

def create_dataset(): 
    enu = create_enu_table()
    hh = create_household_table()
    nut = create_nutrients_table()
    snap = nut.merge(enu, left_on="hhnum", right_on="HHNUM", how="left").merge(
    hh, left_on="hhnum", right_on="hhnum", how="left")

    return snap

def prepare_data():
    snap = create_dataset()
    snap["total_enus"] = snap["weekly_enu"] + snap["guest_enu"]
    snap["total_enus"] = snap["total_enus"] * snap["AME"]

    # drop rows with total_enu == 0, meaning no meals at home were eaten
    snap = snap[snap["total_enus"] != 0]


    # engineer explanatory variables
    snap["household_recieves_snap"] = (snap["snapnowhh"] > 0).astype("int")
    snap["household_weekly_snap_benefits"] = snap["SNAPLASTAMT"]/snap["AME"]
    snap["household_weekly_money_income"] = snap["inchhavg_r"]/snap["AME"]
    snap["household_weekly_money_income_squared"] = snap["household_weekly_money_income"]**2
    snap["school_breakfast"] = (snap["schservebrkfst"] > 0).astype("int")
    snap["houshold_recieves_wic"] = (snap["wichh"] > 0).astype("int")
    snap["female_head_present"] = (snap["female_head"])
    snap["black"] = (snap.Race == "Black").astype("int")
    snap["hispanic"] = (snap.Hispanic == "Hispanic").astype("int")
    snap["guest_meals"] = snap["guest_enu"] / snap["AME"]
    return snap 

In [186]:
snap = prepare_data()

explanatory_vars_mpc = [
    "household_weekly_snap_benefits",
    "household_weekly_money_income",
    "household_weekly_money_income_squared",
    "school_breakfast",
    "houshold_recieves_wic",
    "female_head_present",
    "black",
    "hispanic",
    "midwest",
    "south", 
    "west",
    "guest_meals",
    "nonmetro"
]

explanatory_vars_binary = [
    "household_recieves_snap",
    "household_weekly_money_income",
    "household_weekly_money_income_squared",
    "school_breakfast",
    "houshold_recieves_wic",
    "female_head_present",
    "black",
    "hispanic",
    "midwest",
    "south", 
    "west",
    "guest_meals",
    "nonmetro"
]

response_vars = [
    nut_var for nut_var in snap.columns if re.match(r"fah_*", nut_var)
]

In [187]:
## dataset restriction 1 - target group 1 & 4
## dataset restriction 2 - target group 1, 2, & 4

        
def star_rating(p_value):
    if p_value < 0.001:
        return "***"
    elif p_value < 0.01:
        return "**"
    elif p_value < 0.05:
        return "*"
    else:
        return ""

def run_results(explanatory_vars, response_vars):
    ## run through regression for each nutrient and save results to a log file
    snap_var = explanatory_vars[0]
    snap_restricted = snap[snap["targetgroup"].isin([1,2,4])]
    X = snap_restricted[explanatory_vars]
    X = sm.add_constant(X)

    for nut_var in response_vars:
        y = snap_restricted[nut_var]/snap_restricted["AME"]
        model = sm.OLS(y, X).fit()

        # print out the response variable the coefficent for the household_weekly_snap_benefits variable
        # the standard error, and the p-value for the household_recieves_snap variable, if the 
        # p-value is less than 0.05 print one star, if less than 0.01 print two stars, if less than
        # 0.001 print three stars

        print(f"{nut_var} \t {model.params[snap_var]:.4f}"
            f"({model.bse[snap_var]:.4f})  \t" 
            f"{model.pvalues[snap_var]:.5f}"
            f"{star_rating(model.pvalues[snap_var])}")

run_results(explanatory_vars_mpc, response_vars)

fah_energy 	 21.3486(3.8615)  	0.00000***
fah_carb 	 2.6242(0.5125)  	0.00000***
fah_dietfiber 	 0.0567(0.0303)  	0.06147
fah_totsug 	 1.5523(0.2803)  	0.00000***
fah_totfat 	 0.9048(0.1923)  	0.00000***
fah_satfat 	 0.3102(0.0569)  	0.00000***
fah_monofat 	 0.3335(0.0710)  	0.00000***
fah_polyfat 	 0.1850(0.0657)  	0.00491**
fah_protein 	 0.6903(0.1202)  	0.00000***
fah_alcohol 	 0.0036(0.0319)  	0.90965
fah_chol 	 2.8034(0.5169)  	0.00000***
fah_sodium 	 40.9713(15.5711)  	0.00857**
fah_vitarae 	 6.0652(1.3959)  	0.00001***
fah_vitb6 	 0.0139(0.0038)  	0.00027***
fah_vitb12 	 0.0713(0.0130)  	0.00000***
fah_vitc 	 0.3905(0.1593)  	0.01430*
fah_iron 	 0.1117(0.0339)  	0.00099***
fah_thiamin 	 0.0110(0.0033)  	0.00095***
fah_riboflavin 	 0.0171(0.0032)  	0.00000***
fah_calcium 	 9.0821(1.6436)  	0.00000***
fah_phosphorus 	 12.1350(2.1181)  	0.00000***
fah_magnes 	 1.7997(0.4772)  	0.00017***


In [188]:
run_results(explanatory_vars_binary, response_vars)

fah_energy 	 3098.7268(1166.5523)  	0.00796**
fah_carb 	 382.8973(154.7123)  	0.01340*
fah_dietfiber 	 -0.4879(9.1199)  	0.95734
fah_totsug 	 231.6652(84.6682)  	0.00627**
fah_totfat 	 138.1690(57.9923)  	0.01728*
fah_satfat 	 39.5882(17.1955)  	0.02141*
fah_monofat 	 51.9995(21.4223)  	0.01529*
fah_polyfat 	 35.2306(19.7706)  	0.07489
fah_protein 	 84.3175(36.3347)  	0.02040*
fah_alcohol 	 -3.1563(9.5993)  	0.74233
fah_chol 	 280.0906(156.2512)  	0.07318
fah_sodium 	 4257.6134(4685.7649)  	0.36365
fah_vitarae 	 407.5955(421.1727)  	0.33327
fah_vitb6 	 2.6376(1.1469)  	0.02155*
fah_vitb12 	 9.1807(3.9220)  	0.01933*
fah_vitc 	 5.1942(47.9382)  	0.91373
fah_iron 	 13.0892(10.1985)  	0.19947
fah_thiamin 	 1.5927(1.0002)  	0.11142
fah_riboflavin 	 2.1416(0.9579)  	0.02548*
fah_calcium 	 924.7572(496.9164)  	0.06288
fah_phosphorus 	 1415.0232(640.5010)  	0.02726*
fah_magnes 	 133.6836(143.8365)  	0.35278


In [189]:
cnx.close()